In [60]:
#importing general needed modules
import requests
import pandas as pd

In [4]:
#govinfo python API wrapper
import govinfo 
print(dir(govinfo))

['GovInfo', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'cachetools', 'datetime', 'pytz', 'scrapelib']


In [5]:
from govinfo import GovInfo
print(dir(GovInfo))

['BASE_URL', '__annotations__', '__attrs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_format_time', '_pages', '_throttle', '_throttled', 'accept_response', 'average_request_time', 'close', 'collections', 'congressional_hearings', 'delete', 'disable_compression', 'get', 'get_adapter', 'get_redirect_target', 'head', 'key_for_request', 'merge_environment_settings', 'mount', 'options', 'patch', 'post', 'prepare_request', 'put', 'rebuild_auth', 'rebuild_method', 'rebuild_proxies', 'request', 'requests_per_minute', 'reset_stats', 'resolve_redirects', 'retry_attempts', 'send', 'should_cache_response', 'should_strip_auth', 'urlretrieve', 

In [6]:
api_key = 'redacted'
gov = GovInfo(api_key = api_key)


In [7]:
import json
# Get a list of all available collections
collections_request = gov.collections()
collections_request_json = json.dumps(collections_request)
collections_request_dict = json.loads(collections_request_json)

for collection in collections_request_dict['collections']:
    print(collection)

{'collectionCode': 'BILLS', 'collectionName': 'Congressional Bills', 'packageCount': 249183, 'granuleCount': None}
{'collectionCode': 'BILLSTATUS', 'collectionName': 'Congressional Bill Status', 'packageCount': 138677, 'granuleCount': None}
{'collectionCode': 'BUDGET', 'collectionName': 'United States Budget', 'packageCount': 337, 'granuleCount': 6759}
{'collectionCode': 'CCAL', 'collectionName': 'Congressional Calendars', 'packageCount': 5199, 'granuleCount': 82915}
{'collectionCode': 'CDIR', 'collectionName': 'Congressional Directory', 'packageCount': 125, 'granuleCount': 15024}
{'collectionCode': 'CDOC', 'collectionName': 'Congressional Documents', 'packageCount': 16433, 'granuleCount': 9379}
{'collectionCode': 'CFR', 'collectionName': 'Code of Federal Regulations', 'packageCount': 5805, 'granuleCount': 6814941}
{'collectionCode': 'CHRG', 'collectionName': 'Congressional Hearings', 'packageCount': 35491, 'granuleCount': 286}
{'collectionCode': 'COMPS', 'collectionName': 'Statutes Co

In [8]:
# Set collection code for federal court cases
court_collection_code = 'USCOURTS'
# This is the start date, as in, when we run the program this it will collect cases from present to this day
start_date = '2022-01-01'
offset_val = 0

# Request to API to get 1000 USCOURTS cases
packages_request = gov.get(f'{gov.BASE_URL}/published/{start_date}?offset={offset_val}&pageSize=1000&collection={court_collection_code}&offsetMark=%2A&api_key={api_key}')
packages_request_dict = packages_request.json()


Getting actual cases from API

In [9]:
# Importing Modules
import zipfile
import urllib.request
import io
import os
import glob
import PyPDF2

In [48]:
from zipfile import BadZipFile
# Function for getting case file of a text, given an API link

def get_case_text(link):
    url = link
    response = requests.get(url)

    try:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            # Get the name of the first file in the zip file
            first_file = zip_file.namelist()[1]

            first_file_contents = zip_file.read(first_file)
            # Create a PDF reader object from the contents of the first file
            pdf_reader = PyPDF2.PdfReader(io.BytesIO(first_file_contents))

            # Get the number of pages in the PDF file
            num_pages = len(pdf_reader.pages)

            all_text = []
             # Extract the text from each page in the PDF file
            for i in range(num_pages):
                # Get the page object
                page = pdf_reader.pages[i]

                # Extract the text from the page
                text = page.extract_text()
                all_text.append(text)
            # If the all_text list has PDF text
            if len(all_text) > 0:
                # Join all strings and return
                new_all_text = ' '.join(all_text)
                return new_all_text
            else: 
                # Return URL to the API result
                return url

    except BadZipFile:
        # Return URL to the API result
        return url
    

In [49]:
# Making DataFrame for future use
data = {'title':[], 'text':[]}
text_df = pd.DataFrame(data) 

In [50]:
text_df

,title,text


In [51]:
# Function to search through given API link and get course case data
def get_court_cases(json, dataframe):
    
    final_list = []
    
    # Sort the initial 1000 results from the JSON
    sorted_packages = sorted(json['packages'], key=lambda x: x['dateIssued'])
    new_sorted_packages = [{'title': package['title'],'id': package['packageId']} for package in sorted_packages if (package['packageId'].split('-')[1] in ['ca1', 'ca2', 'ca3', 'ca4', 'ca5', 'ca6', 'ca7', 'ca8', 'ca9', 'ca10', 'ca1', 'ca13'])]
    
    # For each item in the JSON
    for package in new_sorted_packages:
        # Connect to API link for the item
        api_link = f"https://api.govinfo.gov/packages/{package['id']}/zip?api_key={api_key}"
        # Add text of PDF from API link to DataFrame.
        dataframe.loc[len(dataframe.index)] = [package['title'], get_case_text(api_link)]
    
    # If there is another page for the time period (ie, another 1000 results)
    if json['nextPage'] is not None:
        # Get new request
        new_request = gov.get(json['nextPage'])
        # Get JSON from the request
        next_item = new_request.json()
        # Recursively call this function
        get_court_cases(next_item, dataframe)
    else:
        return None

        

Call function

In [52]:
get_court_cases(packages_request_dict, text_df)

unknown widths : 
[0, IndirectObject(167, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(170, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(173, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(176, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(179, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(182, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(185, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(188, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(191, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(195, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(198, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(201, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(204, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(211, 0, 2735440260400)]
unknown widths : 
[0, IndirectObject(95, 0, 2735541619248)]
unknown widths : 
[0, IndirectObject(99, 0, 2735541619248)]
unknown widths : 
[0, Indi

Given CSV of all Federal Court cases from Jan 1, 2022 to today April 7, 2023

In [53]:
text_df

,title,text
0,"Pharmaceutical Research v. Stuart Williams, et al",United States Court of Appeals \nFor The Eigh...
1,"Mirabella v. Town of Lexington, MA, et al",United States Court of Appeals \nFor the Fir...
2,"Andrade-Prado, Jr. v. Garland",United States Court of Appeals \nFor the Fir...
3,USA v. Jose Hodge,NOT PRECEDENTIAL \n \nUNITED STATES COURT OF A...
4,W. R. R. v. Attorney General United States,\n NOT PRECEDENTIAL \n \nUNITED STATES COURT...
...,...,...
16593,"Nicholas Stover v. Amazon.com, LLC, et al",\n \nNOT RECOMMENDED FOR PUBLICATION \nFile ...
16594,"Randy Coley v. DIRECTV, Inc.",PUBLISHED \n \nUNITED STATES COURT OF APPEALS...
16595,US v. Wincy Joseph,UNPUBLISHED \n \nUNITED STATES COURT OF APPEA...
16596,US v. Antonio Davis,UNPUBLISHED \n \nUNITED STATES COURT OF APPEA...


In [54]:
# Save to CSV
text_df.to_csv('C:\\Users\\mjjyo\\OneDrive\\Desktop\\precedent\\2022_federal_cases.csv')

In [59]:
# text_df.loc[16597]['text']